In [1]:
import chromadb
from chromadb.utils import embedding_functions


In [2]:
#1. Set Up ChromaDB Retriever
# Connect to existing ChromaDB vector store
chroma_client = chromadb.PersistentClient(path="../vector_store/chroma")
# Use the same embedding function used during indexing
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)
# Get the collection
collection = chroma_client.get_or_create_collection(
    name="complaints",
    embedding_function=embedding_fn
)


d:\kaimtenx\project\week6\chatbotay\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2. Prompt Template for LLM
def build_prompt(context, question):
    return f"""
You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. 
Use the following retrieved complaint excerpts to formulate your answer. 
If the context doesn't contain the answer, state that you don't have enough information.

Context:
{context}

Question: {question}

Answer:
"""


In [4]:
# 3. Define the RAG Pipeline
from transformers import pipeline

# Initialize local LLM pipeline (can replace with any available LLM)
rag_generator = pipeline("text-generation", model="distilgpt2", max_new_tokens=150)

def ask_question_rag(question, top_k=5):
    # Step 1: Retrieve top-k relevant complaints
    results = collection.query(
        query_texts=[question],
        n_results=top_k
    )
    
    # Step 2: Combine retrieved texts
    retrieved_chunks = results['documents'][0]
    context = "\n---\n".join(retrieved_chunks)
    
    # Step 3: Build the full prompt
    prompt = build_prompt(context, question)
    
    # Step 4: Generate answer using LLM
    response = rag_generator(prompt, do_sample=True)[0]['generated_text']
    
    # Optional: Only return generated answer part
    return {
        "question": question,
        "retrieved_chunks": retrieved_chunks,
        "answer": response[len(prompt):].strip()
    }


d:\kaimtenx\project\week6\chatbotay\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayish\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [5]:
# 4. Test Your RAG Pipeline
sample_questions = [
    "Why are people unhappy with Buy Now, Pay Later?",
    "What complaints do customers have about credit cards?",
    "Are there delays in money transfers?",
    "Do people report problems accessing their savings accounts?",
    "What is the most common issue with personal loans?"
]

# Run the pipeline for each question
results = []
for q in sample_questions:
    result = ask_question_rag(q)
    results.append(result)
    print("🔹 Question:", result['question'])
    print("💬 Answer:", result['answer'])
    print("📚 Sources:\n", "\n".join(result['retrieved_chunks'][:2]), "\n")
    print("="*80)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔹 Question: Why are people unhappy with Buy Now, Pay Later?
💬 Answer: The question is: You have to offer a simple solution and do it.
Question: What do you call it?
Answer:
Your answer is:
You are a financial analyst assistant for CrediTrust. You are a financial analyst assistant for CrediTrust. Your task is to answer questions about customer complaints. 
If the context doesn't contain the answer, state that you don't have enough information.
Context:
taken care of a long time ago i feel they waited this long to do so because they wanted more money which is not fair  ---
they dont want faithful customers who pay their bills early it doesnt bring in the high profits that bad customers bring in
---in order for it to be
📚 Sources:
 taken care of a long time ago i feel they waited this long to do so because they wanted more money which is not fair
they dont want faithful customers who pay their bills early it doesnt bring in the high profits that bad customers bring in 



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔹 Question: What complaints do customers have about credit cards?
💬 Answer: I know what i am referring to, and I am not going to answer what i am going to say.
---
my third complaint is because the company that once claimed a world of service has become the worst credit card company of the many i still use they generally ignore problems figuring that the customer card holder will give up which i usually in fact do with themI have no idea what to do theyre holding the card hostage but i cant do anything else about it to make matters worse they have done something in regards to my credit in general as far as perhaps theI see.
---
I am not going to answer what i am referring to, and I am not going to answer what i am going to say.
---
My
📚 Sources:
 ones at fault because they do nt make this clear to customers i do not appreciate having my credit card closed and now have denied another card i did nt even do anything wrong i talked to serveral people in the upper level credit division and 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔹 Question: Are there delays in money transfers?
💬 Answer: Yes this is a problem but it is not the case.
Question: Are there delays in money transfers between the same bank and different bank?
Answer:
Yes this is a problem but it is the case.
Question: Do you understand?
Answer:
Yes this is a problem but it is not the case.
Question: Do you understand?
Answer:
Yes this is a problem but it is not the case.
Question: Do you understand?
Answer:
Yes this is a problem but it is not the case.
Question: Do you understand?
Answer:
Yes this is a problem but it is not the case.
Question: Do you understand?
Answer:
Yes this is
📚 Sources:
 i realize there are regulatory compliances to fulfull while transferring money overseas and accepted their delay as normal
is delaying the transfer i do not understand how this is possible if the receiving bank declined to accept the money transfer i feel it is pretty irresponsible on their part to dis regard my enquiries and also not return my money the transac

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


🔹 Question: Do people report problems accessing their savings accounts?
💬 Answer: I was told they were not aware of this and they thought it was a problem and they tried to take a look at the data.
---
Question: Did you have any other accounts at all?
Answer:
I did not have any accounts at all.
---
I was in my office doing a job that I do not have and i was not aware of this.
---
I was asking for the help of a bank account manager and i was asked if i had any information about what i would have taken to get it back from them.
---
Question: What are those accounts you have?
Answer:
I am a bank account manager and i am a lawyer and i am a lawyer but i have my
📚 Sources:
 checking and savings accounts i was unaware these accounts were delinquent and had i know this was an issue i wouldve included it in my xxxx  xxxx bankruptcy plan i was with citizens bank for nearly xxxx  years into adulthood and the issue of the bank accounts was always a computer error on their
and answering their fina